In [27]:
import pandas as pd
import numpy as np

In [25]:
bill = pd.read_csv('Grocery_Bill.csv')
bill.head(3)

,Item_Cost,Joy,Aurora,Silas,Adam,Diego
0,2.49,x,x,NaN,NaN,NaN
1,1.34,NaN,x,x,x,x
2,1.33,NaN,x,x,NaN,NaN


In [26]:
bill['Cost_Per_Person'] = bill['Item_Cost'] / (bill.count(axis=1) - 1)
bill.head(3)

,Item_Cost,Joy,Aurora,Silas,Adam,Diego,Cost_Per_Person
0,2.49,x,x,NaN,NaN,NaN,1.245
1,1.34,NaN,x,x,x,x,0.335
2,1.33,NaN,x,x,NaN,NaN,0.665


In [36]:
def get_costs(li):
    costs = []
    for i in li:
        cost = bill.groupby(i).sum().Cost_Per_Person[0]
        costs.append(cost)
    return costs

In [51]:
even_split = pd.DataFrame({'Person':bill.columns[1:-1],'Even_Cost':get_costs(bill.columns[1:-1])})
even_split.head()

,Person,Even_Cost
0,Joy,19.243333
1,Aurora,16.971667
2,Silas,20.425833
3,Adam,15.521667
4,Diego,25.417500


In [98]:
amounts = []
for i in even_split.Person:
    amount = input('How much does ' + i + ' eat? (Small, Medium, or Large) ').capitalize()
    amounts.append(amount)
even_split['Amount'] = amounts

How much does Joy eat? (Small, Medium, or Large) small
How much does Aurora eat? (Small, Medium, or Large) medium
How much does Silas eat? (Small, Medium, or Large) large
How much does Adam eat? (Small, Medium, or Large) medium
How much does Diego eat? (Small, Medium, or Large) large


In [99]:
even_split

,Person,Even_Cost,Amount,Modifier,Modified_Cost
0,Joy,19.243333,Small,0.87,16.741700
1,Aurora,16.971667,Medium,0.78,13.237900
2,Silas,20.425833,Large,1.18,24.102483
3,Adam,15.521667,Medium,0.87,13.503850
4,Diego,25.417500,Large,1.18,29.992650


In [63]:
total = even_split.Even_Cost.sum()
large_percentages = np.arange(1.000, 1.300, .005)
medium_percentages = np.arange(.800, 1.000, .005)
small_percentages = np.arange(.700, .800, .005)

In [76]:
large_numbers = []
medium_numbers = []
small_numbers = []
modified_totals = []

large_total = even_split[even_split.Amount == 'Large'].groupby('Amount').sum().Even_Cost[0]
medium_total = even_split[even_split.Amount == 'Medium'].groupby('Amount').sum().Even_Cost[0]
small_total = even_split[even_split.Amount == 'Small'].groupby('Amount').sum().Even_Cost[0]

for i in large_percentages:
    large_modified = large_total * i
    for j in medium_percentages:
        medium_modified = medium_total * j
        for l in small_percentages:
            small_modified = small_total * l
            
            modified_total = large_modified + medium_modified + small_modified
            modified_totals.append(modified_total)
            large_numbers.append(i)
            medium_numbers.append(j)
            small_numbers.append(l)

In [77]:
distances = abs(total-modified_totals)
    
multipliers = np.array(large_numbers) / np.array(medium_numbers)
    
multipliers2 = np.array(large_numbers) / np.array(small_numbers)
    
multipliers3 = np.array(medium_numbers) / np.array(small_numbers)

In [84]:
d = {'Large_Modifier':large_numbers, 'Medium_Modifier':medium_numbers, 'Small_Modifier':small_numbers, 'Modified_Totals':modified_totals, 'Absolute_Distance':distances, 'Large:Medium':multipliers, 'Large:Small':multipliers2, 'Medium:Small':multipliers3}
Pareto_Table = pd.DataFrame(data=d)
selection = Pareto_Table[(Pareto_Table.Absolute_Distance < .01)].sort_values(['Absolute_Distance']).head(20).reset_index().drop(['index', 'Absolute_Distance', 'Modified_Totals'], axis=1)
selection

,Large_Modifier,Medium_Modifier,Small_Modifier,Large-Medium,Large-Small,Medium-Small
0,1.225,0.840,0.720,1.458333,1.701389,1.166667
1,1.140,0.935,0.755,1.219251,1.509934,1.238411
2,1.125,0.945,0.775,1.190476,1.451613,1.219355
3,1.210,0.850,0.740,1.423529,1.635135,1.148649
4,1.240,0.830,0.700,1.493976,1.771429,1.185714
5,1.155,0.925,0.735,1.248649,1.571429,1.258503
6,1.110,0.955,0.795,1.162304,1.396226,1.201258
7,1.195,0.860,0.760,1.389535,1.572368,1.131579
8,1.220,0.810,0.795,1.506173,1.534591,1.018868
9,1.130,0.975,0.700,1.158974,1.614286,1.392857


In [87]:
select = int(input('Select which modifiers to use to split the grocery bill by inputting the index of your preferred row: '))

Select which modifiers to use to split the grocery bill by inputting the index of your preferred row: 11


In [89]:
modifiers = selection.iloc[select][0:3]
modifiers

Large_Modifier     1.18
Medium_Modifier    0.87
Small_Modifier     0.78
Name: 11, dtype: float64

In [93]:
even_split['Modifier'] = even_split.Amount.map({'Large': modifiers[0], 'Medium': modifiers[1], 'Small': modifiers[2]})

In [96]:
even_split['Modified_Cost'] = even_split.Even_Cost * even_split.Modifier

In [97]:
final_cost = even_split
final_cost

,Person,Even_Cost,Amount,Modifier,Modified_Cost
0,Joy,19.243333,Medium,0.87,16.741700
1,Aurora,16.971667,Small,0.78,13.237900
2,Silas,20.425833,Large,1.18,24.102483
3,Adam,15.521667,Medium,0.87,13.503850
4,Diego,25.417500,Large,1.18,29.992650
